# Bol.com Web Scraping

In [ ]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

from tqdm import tqdm

In [ ]:
categories = ['laptops', 'desktops', 'monitoren', 'randapparatuur', 'netwerk-&-internet', 'tassen-&-hoezen', 'printers-&-supplies', 'pc-gaming-winkel', 'software', 'dataopslag', 'beamers', 'componenten']

In [ ]:
print(len(categories))

In [ ]:
all_products = []

for i in (categories):
    for j in tqdm(range(1, 501)):
        response = requests.get(f'https://www.bol.com/nl/s/?page={j}&searchtext={i}&view=list').content
        soup = BeautifulSoup(response, 'lxml')
        products = soup.find_all(class_='product-item--row js_item_root')
        all_products.extend(products)
        time.sleep(2)

In [ ]:
lst = []

for item in all_products:
    if item.find(class_='product-prices').getText() == '\nNiet leverbaar\n':
        price = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn nog geen beoordelingen voor dit product.':
        score = None
    elif item.find(class_='star-rating')['title'] == 'Er zijn  nog geen beoordelingen voor dit product.':
        review = None
    else:
        product = item.find(class_='product-title--inline').a.getText()
        price = float(item.meta['content'])
        score = item.find(class_='star-rating')['title']
        review = item.find(class_='star-rating')['data-count']
        lst.append([product, price, score, review])

In [ ]:
print(lst[:10])

In [ ]:
products_downloaded = len(lst)

In [ ]:
df = pd.DataFrame(lst, columns=['Product', 'Price', 'Score', 'Reviews'])

In [ ]:
len(df)

In [ ]:
df['Price'] = df['Price'].dropna()

In [ ]:
len(df)

In [ ]:
df = df.drop_duplicates()

In [ ]:
len(df)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df['Score'] = df['Score'].str.strip('klantbeoordeling: ')

In [ ]:
df['Score'] = df['Score'].str[:3]

In [ ]:
df['Score'] = df['Score'].astype(float)

In [ ]:
df['Reviews'] = df['Reviews'].astype(int)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
print('Products Downloaded:', products_downloaded, '\nDataset Size:', len(df))

In [ ]:
df.to_csv('../data/processed/bol.csv')